In [1]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import glob
import io
from collections import namedtuple
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

### Set FilePath

In [2]:
LABEL_PATH = r"F:\Minor Project\Monument Object Detection\helper\Report\ps-up-sp-BDS-alamy-merged.csv"
ignore_list = ['kotilingeshvara', 'til mahadev narayan temple', 'king statue']
mistake_list = ['degutale temple', 'kritipur tower']
correct_list = ['degu tale temple_KDS', 'kirtipur tower']

In [3]:
POSSIBLE_LABELS = dict()

In [4]:
def csv_to_label_map_dict(path):
    df = pd.read_csv(path)
    class_list = list(df['class'])
    final_class_list = list()
    for class_name in class_list:
        if class_name not in ignore_list and class_name not in correct_list:
            if class_name in mistake_list:
                if class_name == 'degutale temple':
                    class_name = correct_list[0]
                elif class_name == 'kritipur tower':
                    class_name = correct_list[1]
            final_class_list.append(class_name)
    class_index = 1
    res_dict = {}
    for class_label in final_class_list:
        res_dict[class_label] = class_index
        class_index += 1
    return res_dict

In [5]:
csv_to_label_map_dict(LABEL_PATH)

{'badrinath temple': 1,
 'basantapur tower': 2,
 'bhagavati temple': 3,
 'bhairavnath temple': 4,
 'bhaktapur tower': 5,
 'bhimeleshvara': 6,
 'bhimsen temple': 7,
 'bhupatindra malla column': 8,
 'bhuvana lakshmeshvara': 9,
 'chasin dega': 10,
 'chayasilin mandap': 11,
 'dattatreya temple': 12,
 'degu tale temple_KDS': 13,
 'fasidega temple': 14,
 'gaddi durbar': 15,
 'garud': 16,
 'golden gate': 17,
 'gopinath krishna temple': 18,
 'hanuman idol': 19,
 'indrapura': 20,
 'jagannatha temple': 21,
 'kala-bhairava': 22,
 'kasthamandap': 23,
 'kavindrapura sattal': 24,
 'kedamatha tirtha': 25,
 'kirtipur tower': 26,
 'kumari ghar': 27,
 'lalitpur tower': 28,
 'mahadev temple': 29,
 'narayan temple': 30,
 'national gallery': 31,
 'nyatapola temple': 32,
 'palace of the 55 windows': 33,
 'panchamukhi hanuman': 34,
 'pratap malla column': 35,
 'shiva temple': 36,
 'shveta bhairava': 37,
 'siddhi lakshmi temple': 38,
 'simha sattal': 39,
 'taleju bell_BDS': 40,
 'taleju bell_KDS': 41,
 'talej

In [6]:
POSSIBLE_LABELS = csv_to_label_map_dict(LABEL_PATH)

In [50]:
def check_class_validity(class_name):
    if class_name == 'degutale temple' or class_name == 'degu tale':
        return 'degu tale temple_KDS'
    elif class_name == 'kritipur tower':
        return 'kirtipur tower'
    elif class_name not in list(POSSIBLE_LABELS.keys()):
        return None
    else:
        return class_name

In [59]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        all_objects = root.findall('object')
        changed_filename = xml_file.split('\\')[-1].split('.')[0] + '.JPG'
        # Handling xml parser for random background images
        if len(all_objects) != 0:
            for member in all_objects:
                class_label = check_class_validity(member[0].text)
                if class_label is not None:
                    value = (changed_filename,
                             int(root.find('size')[0].text),
                             int(root.find('size')[1].text),
                             class_label,
                             int(member[4][0].text),
                             int(member[4][1].text),
                             int(member[4][2].text),
                             int(member[4][3].text)
                             )
                    xml_list.append(value)
        else:
            # Setting background class to 'bg', and four bbox coordinates all to neg 1.
            value = (changed_filename,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     'bg',
                      -1,
                      -1,
                      -1,
                      -1)
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [60]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [61]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [62]:
def class_text_to_int(row_label):
    return POSSIBLE_LABELS[row_label]

In [63]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in tqdm(group.object.iterrows()):
        if int(row['xmin']) > -1:
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(class_text_to_int(row['class']))
            
        else:
            xmins.append(0)
            xmaxs.append(0)
            ymins.append(0)
            ymaxs.append(0)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(0)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_jpg),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example

In [64]:
def TFRecord_Creator(image_dir, annotations_dir, output_file, split_type = "BDG"):
    writer = tf.io.TFRecordWriter(output_file)
    path = os.path.join(image_dir)
    examples = xml_to_csv(annotations_dir)
    #grouped by the filename so that multiple monument detection boxes are grouped in a single 'GROUP'
    grouped = split(examples, 'filename') 
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file for ', split_type)

In [65]:
BASE_DIR = "F:\Minor Data Collection\Final Image Data\Monument Original\Merged_Original_Augmented"
annotations_path = 'Annotations'
jpg_images_path = 'JPEGImages'
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')
test_dir = os.path.join(BASE_DIR, 'test')

In [67]:
TFRecord_Creator(os.path.join(train_dir, jpg_images_path), os.path.join(train_dir, annotations_path), os.path.join(BASE_DIR, 'train.tfrecord'))
TFRecord_Creator(os.path.join(test_dir, jpg_images_path), os.path.join(test_dir, annotations_path), os.path.join(BASE_DIR, 'test.tfrecord'))

2it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 999.60it/s]
1it [00:00, 1050.15it/s]
1it [00:00, 1004.14it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1003.42it/s]
3it [00:00, ?it/s]
3it [00:00, 3000.22it/s]
3it [00:00, 3008.83it/s]
3it [00:00, ?it/s]
3it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, 1002.82it/s]
3it [00:00, 1505.49it/s]
3it [00:00, ?it/s]
3it [00:00, 10932.16it/s]
3it [00:00, 3009.55it/s]
3it [00:00, 2929.67it/s]
2it [00:00, 2015.04it/s]
2it [00:00, 2004.45it/s]
2it [00:00, 1993.49it/s]
2it [00:00, 2008.29it/s]
2it [00:00, 2005.40it/s]
2it [00:00, 2016.01it/s]
2it [00:00, ?it/s]
2it [00:00, 2002.53it/s]
2it [00:00, 2006.84it/s]
1it [00:00, 1001.98it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.55it/s]
1it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, 2051.51it/s]
2it [00:00, ?it/s]
2it [00:

1it [00:00, ?it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1001.03it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.07it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, 3010.99it/s]
3it [00:00, ?it/s]
3it [00:00, ?it/s]
3it [00:00, 3010.99it/s]
3it [00:00, 1532.26it/s]
3it [00:00, 3141.02it/s]
3it [00:00, 3009.55it/s]
3it [00:00, 3011.71it/s]
3it [00:00, 3008.11it/s]
3it [00:00, 1501.00it/s]
1it [00:00, 999.36it/s]
1it [00:00, 3758.34it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 998.64it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.51it/s]
1it [00:00, ?it/s]
1it [00:00, 1001.51it/s]
1it [00:00, 1047.53it/s]
1it [00:00, 1000.07it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 999.83it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, 3006.67it/s]
3it [00:00, 1501.54it/s]
3it [00:00, 3000.22it/s]
3it [00:00, 2885.99it/s]
3it [00:00, 3046.71it/s]
3it [00:00, 4355.46it/s]
3it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
1it [

1it [00:00, 1003.66it/s]
1it [00:00, 1015.08it/s]
1it [00:00, 1001.27it/s]
1it [00:00, 1002.46it/s]
1it [00:00, 1000.07it/s]
1it [00:00, 996.04it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1006.79it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 502.55it/s]
1it [00:00, 780.05it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1941.81it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.79it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1006.55it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 198.79it/s]
2it [00:00, ?it/s]
1it [00:00, 1001.74it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 999.60it/s]
1it [00:00, ?it/s]
2it [00:00, 989.34it/s]
1it [00:00, 497.96it/s]
1it [00:00, 2439.97it/s]
1it [00:00, ?it/s]
1it [00:00, 332.01it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 100.44it/s]
2it [00:00, 1893.16it/s]
2it [00:00, 672.87it/s]
2it [00:00, ?it/s]
2it [00:00, 1954.02it/s]
2it [00:00, 1929.30it/s]
2it [00:00, 

3it [00:00, 3010.99it/s]
2it [00:00, 1998.24it/s]
2it [00:00, 2054.02it/s]
2it [00:00, ?it/s]
2it [00:00, ?it/s]
4it [00:00, 4009.85it/s]
4it [00:00, 4019.46it/s]
4it [00:00, ?it/s]
4it [00:00, ?it/s]
4it [00:00, ?it/s]
4it [00:00, 4006.02it/s]
4it [00:00, 3992.67it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.18it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, 2010.69it/s]
5it [00:00, 2762.68it/s]
5it [00:00, 2501.67it/s]
5it [00:00, ?it/s]
4it [00:00, 3987.93it/s]
4it [00:00, 1003.48it/s]
4it [00:00, 4011.77it/s]
2it [00:00, ?it/s]
2it [00:00, 2000.62it/s]
2it [00:00, 1009.46it/s]
3it [00:00, 2998.79it/s]
3it [00:00, ?it/s]
3it [00:00, 3012.43it/s]
1it [00:00, 2336.66it/s]
1it [00:00, 1000.55it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1001.03it/s]
3it [00:00, 2998.79it/s]
3it [00:00, 3011.71it/s]
3it [00:00, ?it/s]
3it [00:00, 3006.67it/s]
3it [00:00, 1502.98it/s]
3it [00:00, 3006.67it/s]
6it [00:00, 6068.44it/s]
6it [00:00, 3649.34it/s]
6it

1it [00:00, 1001.51it/s]
2it [00:00, 1976.11it/s]
2it [00:00, 2006.36it/s]
2it [00:00, 987.01it/s]
5it [00:00, 5014.71it/s]
5it [00:00, 5007.53it/s]
5it [00:00, 2501.08it/s]
5it [00:00, 5020.71it/s]
5it [00:00, 5014.71it/s]
2it [00:00, 2004.45it/s]
5it [00:00, 5019.51it/s]
5it [00:00, 5023.12it/s]
2it [00:00, 2009.25it/s]
1it [00:00, 1001.74it/s]
1it [00:00, 1010.92it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1001.74it/s]
4it [00:00, 3989.83it/s]
4it [00:00, 2006.60it/s]
4it [00:00, 2002.05it/s]
4it [00:00, 11781.75it/s]
4it [00:00, ?it/s]
4it [00:00, 4025.24it/s]
4it [00:00, 641.77it/s]
4it [00:00, 1339.50it/s]
1it [00:00, ?it/s]
4it [00:00, 3977.53it/s]
6it [00:00, 6013.34it/s]
6it [00:00, 6016.21it/s]
6it [00:00, 6020.53it/s]
6it [00:00, 5976.21it/s]
6it [00:00, 6052.39it/s]
2it [00:00, 2016.49it/s]
3it [00:00, 3011.71it/s]
1it [00:00, 999.60it/s]
1it [00:00, 4373.62it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1003.90it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 994.85it/s]
1i

...

1it [00:00, ?it/s]
2it [00:00, 1002.10it/s]
2it [00:00, 1492.63it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 2001.58it/s]
1it [00:00, 504.24it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 2103.46it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 2004.93it/s]
2it [00:00, 2001.58it/s]
2it [00:00, 1931.08it/s]
2it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, 1701.89it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
2it [00:00, 2007.80it/s]
2it [00:00, 2014.56it/s]
2it [00:00, ?it/s]
1it [00:00, 1001.98it/s]
1it [00:00, 1001.03it/s]
1it [00:00, 999.36it/s]
2it [00:00, 2016.49it/s]
2it [00:00, 2000.62it/s]
2it [00:00, 2010.21it/s]
2it [00:00, 1927.53it/s]
3it [00:00, ?it/s]
1it [00:00, 1701.54it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1004.14it/s]
1it [00:00, 1002.46it/s]
2it [00:00, ?it/s]
2it [00:00, 2003.49it/s]
1it [00:00, 1000.55it/s]
1it [00:00, ?it/s]
2it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 961.78it/s]

2it [00:00, 2000.14it/s]
4it [00:00, 4003.15it/s]
2it [00:00, ?it/s]
4it [00:00, 4020.42it/s]
3it [00:00, 3003.80it/s]
3it [00:00, 3006.67it/s]
3it [00:00, 3123.09it/s]
3it [00:00, ?it/s]
4it [00:00, 4006.02it/s]
4it [00:00, 3871.07it/s]
3it [00:00, 2020.70it/s]
3it [00:00, ?it/s]
4it [00:00, 3995.53it/s]
4it [00:00, 4032.02it/s]
4it [00:00, 2041.52it/s]
3it [00:00, 3117.67it/s]
2it [00:00, 1925.76it/s]
2it [00:00, 2078.96it/s]
1it [00:00, 973.61it/s]
1it [00:00, 996.51it/s]
3it [00:00, 2994.51it/s]
3it [00:00, 3005.23it/s]
1it [00:00, 1003.42it/s]
1it [00:00, 1003.42it/s]
2it [00:00, 400.37it/s]
1it [00:00, 1000.79it/s]
1it [00:00, 1001.51it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1004.14it/s]
1it [00:00, 1003.18it/s]
1it [00:00, ?it/s]
1it [00:00, 501.11it/s]
1it [00:00, 499.02it/s]
1it [00:00, 1003.42it/s]
1it [00:00, 503.22it/s]
1it [00:00, 1003.18it/s]
1it [00:00, 1003.66it/s]
1it [00:00, 1000.79it/s]
2it [00:00, 665.66it/s]
2it [00:00, 1002.22it/s]
1it [00:00, 1002.70it/s]
1it [0

1it [00:00, ?it/s]
2it [00:00, 1296.34it/s]
1it [00:00, 1002.22it/s]
1it [00:00, 1887.63it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1003.66it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
3it [00:00, ?it/s]
1it [00:00, 1001.51it/s]
1it [00:00, ?it/s]
2it [00:00, 2088.80it/s]
2it [00:00, 2000.62it/s]
1it [00:00, 995.56it/s]
2it [00:00, ?it/s]
2it [00:00, 1937.32it/s]
1it [00:00, 1042.06it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 980.89it/s]
1it [00:00, 1051.47it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.22it/s]
2it [00:00, ?it/s]
2it [00:00, 1419.15it/s]
2it [00:00, 2706.88it/s]
1it [00:00, ?it/s]
2it [00:00, ?it/s]
1it [00:00, 1002.46it/s]
2it [00:00, ?it/s]
3it [00:00, 3018.93it/s]
2it [00:00, 1999.67it/s]
2it [00:00, 2007.80it/s]
2it [00:00, ?it/s]
2it [00:00, ?it/s]
1it [00:00, ?it/s]
2it [00:00, ?it/s]
2it [00:00, 1993.49it/s]
3it [00:00, 2877.41it/s]
3it [00:00, 3006.67it/s]
3it [00:00, 290

1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1000.07it/s]
1it [00:00, 1005.11it/s]
1it [00:00, 1002.70it/s]
1it [00:00, 1002.70it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 996.51it/s]
1it [00:00, 1004.86it/s]
1it [00:00, 497.84it/s]
1it [00:00, 1001.27it/s]
1it [00:00, ?it/s]
1it [00:00, 942.33it/s]
1it [00:00, 963.32it/s]
1it [00:00, 1009.46it/s]
1it [00:00, 968.89it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 1002.94it/s]
1it [00:00, 1007.04it/s]
1it [00:00, ?it/s]
1it [00:00, 1018.78it/s]
1it [00:00, 970.01it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 968.89it/s]
2it [00:00, 2005.88it/s]
1it [00:00, 998.41it/s]
1it [00:00, 996.04it/s]
1it [00:00, 999.60it/s]
1it [00:00, 1039.48it/s]
1it [00:00, 1038.45it/s]
1it [00:00, 996.98it/s]
1it [00:00, 992.97it/s]
2it [00:00, ?it/s]
2it [00:00, 2005.88it/s]
2it [00:00, 2871.83it/s]
1it [00:00, ?it/s]
1it [00:00, ?it/s]
1it [00:00, 631.67it/s]
1it [00:00, 1979.38it/s]
1it [00:00, ?it/s]
1it

Successfully created the TFRecord file for  BDG


In [74]:
# image_dir = './Dataset/test/JPEGImages'
# annotations_dir = './Dataset/test/Annotations'
# output_file = './monument-test.tfrecord'
# main(image_dir, annotations_dir, output_file)

In [ ]:
# def xml_to_csv(path):
#     xml_list = []
#     for xml_file in glob.glob(path + '/*.xml'):
#         tree = ET.parse(xml_file)
#         root = tree.getroot()
#         for member in root.findall('object'):
#             class_label = check_class_validity(member[0].text)
#             if class_label is not None:
#                 value = (root.find('filename').text,
#                          int(root.find('size')[0].text),
#                          int(root.find('size')[1].text),
#                          class_label,
#                          int(member[4][0].text),
#                          int(member[4][1].text),
#                          int(member[4][2].text),
#                          int(member[4][3].text)
#                          )
#                 xml_list.append(value)
#     column_name = ['filename', 'width', 'height',
#                    'class', 'xmin', 'ymin', 'xmax', 'ymax']
#     xml_df = pd.DataFrame(xml_list, columns=column_name)
#     return xml_df